# Debug: Replica Bidirezionale Google Drive PouchDB Adapter

## Problema
La replica `localDB.replicate.to(googleDriveDB)` funziona ✅  
Ma `localDB.replicate.from(googleDriveDB)` non sincronizza i dati ❌

Questo notebook diagnostica il flusso di replicazione per identificare dove il blocco si verifica.

## 1️⃣ Section: Setup e Configurazione Replica

### Aggiungi questo codice al tuo componente per diagnosticare il problema:

In [ ]:
// Setup Google Drive Adapter con debug logging
const plugin = GoogleDriveAdapter({
    accessToken: async () => driveConfig.accessToken,
    folderName: 'my-db',
    pollingIntervalMs: 5000,
    debug: true  // 🔍 Abilita logging dettagliato
});

PouchDB.plugin(plugin);

const googleDriveDB = new PouchDB('paper-drive-db', {
    adapter: 'googledrive'
});

const localDB = stack?.db;

// 🔍 Aggiungi listener per monitorare gli eventi di replicazione
console.log('=== SETUP REPLICA BIDIREZIONALE ===');

// Replica TO (localDB → googleDriveDB) - Funzionante ✅
const repTo = localDB.replicate.to(googleDriveDB, { live: true, retry: true });
repTo.on('change', (info) => {
    console.log('📤 TO: change event', { 
        direction: 'local→gdrive',
        docs: info.docs?.length,
        ok: info.ok
    });
});
repTo.on('complete', (info) => {
    console.log('✅ TO: replication complete', info);
});
repTo.on('error', (err) => {
    console.error('❌ TO: replication error', err);
});

// Replica FROM (googleDriveDB → localDB) - Problema ❌
const repFrom = localDB.replicate.from(googleDriveDB, { live: true, retry: true });
repFrom.on('change', (info) => {
    console.log('📥 FROM: change event', { 
        direction: 'gdrive→local',
        docs: info.docs?.length,
        ok: info.ok
    });
});
repFrom.on('complete', (info) => {
    console.log('✅ FROM: replication complete', info);
});
repFrom.on('error', (err) => {
    console.error('❌ FROM: replication error', err);
});

## 2️⃣ Section: Diagnosi - Verifica Flusso di Replicazione

### Controlla le sequenze e i metadati del database

In [ ]:
// Confronta lo stato dei due database
async function compareDatabases() {
    const localInfo = await localDB.info();
    const googleDriveInfo = await googleDriveDB.info();
    
    console.log('📊 DATABASE INFO COMPARISON:');
    console.log('Local DB:', {
        doc_count: localInfo.doc_count,
        update_seq: localInfo.update_seq,
        db_name: localInfo.db_name
    });
    console.log('Google Drive DB:', {
        doc_count: googleDriveInfo.doc_count,
        update_seq: googleDriveInfo.update_seq,
        db_name: googleDriveInfo.db_name
    });
    
    return { localInfo, googleDriveInfo };
}

// Esegui la diagnosi dopo 25 secondi (dopo che il polling ha avuto tempo di rilevare)
setTimeout(async () => {
    console.log('\n=== DIAGNOSI DOPO 25 SEC ===\n');
    
    const { localInfo, googleDriveInfo } = await compareDatabases();
    
    // 🔍 Verifica se le sequenze sono sincronizzate
    if (localInfo.update_seq === googleDriveInfo.update_seq) {
        console.log('✅ Sequenze sincronizzate');
    } else {
        console.warn('⚠️  Sequenze NON sincronizzate:', {
            local: localInfo.update_seq,
            googleDrive: googleDriveInfo.update_seq,
            diff: googleDriveInfo.update_seq - localInfo.update_seq
        });
    }
    
    // 🔍 Verifica il documento specifico su Google Drive
    try {
        const docOnDrive = await googleDriveDB.get('Notebook-0');
        console.log('✅ Notebook-0 trovato su Google Drive:', {
            _id: docOnDrive._id,
            _rev: docOnDrive._rev
        });
    } catch (err) {
        console.error('❌ Notebook-0 NON trovato su Google Drive:', err.message);
    }
    
    // 🔍 Verifica il documento su Local DB
    try {
        const docLocal = await localDB.get('Notebook-0');
        console.log('✅ Notebook-0 trovato su Local DB:', {
            _id: docLocal._id,
            _rev: docLocal._rev
        });
    } catch (err) {
        console.error('❌ Notebook-0 NON trovato su Local DB:', err.message);
    }
    
}, 25000);

## 3️⃣ Section: Debug - Polling e Change Detection

### Il problema probabilmente è nel meccanismo di Polling

Verifica se il polling sta effettivamente rilevando i cambiamenti da Google Drive:

In [ ]:
// 🔍 Monitora i cambiamenti rilevati tramite _changes API
async function debugChangesAPI() {
    console.log('\n=== TESTING _CHANGES API ===\n');
    
    // Ottieni gli ultimi 10 cambiamenti dal Google Drive DB
    try {
        const changes = await googleDriveDB.changes({
            include_docs: true,
            descending: true,
            limit: 10
        });
        
        console.log('Recent changes from Google Drive DB:', {
            results: changes.results.length,
            last_seq: changes.last_seq,
            results: changes.results.map(r => ({
                id: r.id,
                seq: r.seq,
                changes: r.changes,
                deleted: r.deleted,
                hasDoc: !!r.doc
            }))
        });
        
        return changes;
    } catch (err) {
        console.error('❌ Error fetching changes from Google Drive DB:', err);
    }
}

// Testa la _changes API subito
await debugChangesAPI();

// Testa di nuovo dopo 10 secondi per vedere se il polling ha rilevato nuovi cambiamenti
setTimeout(async () => {
    console.log('\n=== TESTING _CHANGES API AFTER POLLING ===\n');
    await debugChangesAPI();
}, 10000);

## 🚨 POSSIBILI CULPRIT IDENTIFICATI

### Problema 1: _changes feed con `include_docs` potrebbe essere lento

Nel tuo `adapter.ts`, quando `opts.include_docs` è vero, il metodo `_changes` deve:
- Scaricare il CORPO di ogni documento (lento)
- Questo blocca la replica

**Diagnosi:**  
Controlla la console del browser. Se vedi molti `fetchFile` calls per ogni change, questo è il collo di bottiglia.

### Problema 2: Change detection non rispetta il `since` parameter

La replica PouchDB invia `since: lastSeq` per ricevere solo i cambiamenti NUOVI.  
Se `_changes` non filtra correttamente, la replica non rileva i nuovi documenti.

### Problema 3: Live listener potrebbe non essere chiamato

Nel `DriveHandler`, il polling rileva modifiche a `_meta.json`, ma:
- Il polling ha un delay di 5 secondi
- La replicazione potrebbe timeout prima di ricevere notifiche

## 4️⃣ Section: Troubleshooting - Conflitti di Revisione

### Verifica validazione revisioni

In [ ]:
// Verifica i conflitti di revisione durante la replica
async function checkReplicationConflicts() {
    console.log('\n=== CHECKING REPLICATION CONFLICTS ===\n');
    
    // Leggi lo stesso documento da entrambi i database
    const docId = 'Notebook-0';
    
    try {
        const localDoc = await localDB.get(docId);
        console.log('Local Doc:', { _id: localDoc._id, _rev: localDoc._rev });
    } catch (e) {
        console.warn('Local doc not found');
    }
    
    try {
        const remoteDoc = await googleDriveDB.get(docId);
        console.log('Remote Doc:', { _id: remoteDoc._id, _rev: remoteDoc._rev });
    } catch (e) {
        console.warn('Remote doc not found');
    }
    
    // 🔍 Verifica _conflicts (documenti con conflitti di merge)
    try {
        const allDocs = await localDB.allDocs({ conflicts: true });
        const conflictedDocs = allDocs.rows.filter(r => r.value.conflicts && r.value.conflicts.length > 0);
        if (conflictedDocs.length > 0) {
            console.warn('⚠️  Found documents with conflicts:', conflictedDocs);
        } else {
            console.log('✅ No conflicts found in local DB');
        }
    } catch (e) {
        console.error('Error checking conflicts:', e);
    }
}

// Esegui il check dopo 30 secondi
setTimeout(async () => {
    await checkReplicationConflicts();
}, 30000);

## 5️⃣ Section: Validazione - Test della Replica Bidirezionale

### Test Manuale: Crea un documento su Google Drive e verifica la sincronizzazione

In [ ]:
// 🧪 Test Completo della Replica Bidirezionale

async function runFullReplicationTest() {
    console.log('\n=== FULL REPLICATION TEST ===\n');
    
    // Step 1: Leggi lo stato iniziale
    const localInfoBefore = await localDB.info();
    const googleDriveInfoBefore = await googleDriveDB.info();
    
    console.log('Initial State:');
    console.log('  Local:', { doc_count: localInfoBefore.doc_count, seq: localInfoBefore.update_seq });
    console.log('  Google Drive:', { doc_count: googleDriveInfoBefore.doc_count, seq: googleDriveInfoBefore.update_seq });
    
    // Step 2: Crea un nuovo documento SU GOOGLE DRIVE
    const testDocId = `test-sync-${Date.now()}`;
    const testDoc = {
        _id: testDocId,
        title: 'Test Document for Replication',
        createdAt: new Date().toISOString(),
        source: 'google-drive-db'
    };
    
    try {
        const saveResult = await googleDriveDB.put(testDoc);
        console.log('\n✅ Created document on Google Drive:', { id: saveResult.id, rev: saveResult.rev });
    } catch (err) {
        console.error('❌ Failed to create document on Google Drive:', err);
        return;
    }
    
    // Step 3: Attendi il polling + replicazione (20 secondi dovrebbe essere sufficiente)
    console.log('\n⏳ Waiting for polling and replication (20 seconds)...');
    
    await new Promise(resolve => setTimeout(resolve, 20000));
    
    // Step 4: Verifica se il documento è arrivato al Local DB
    try {
        const docOnLocal = await localDB.get(testDocId);
        console.log('\n✅✅✅ SUCCESS! Document replicated to local DB:', {
            id: docOnLocal._id,
            rev: docOnLocal._rev,
            title: docOnLocal.title
        });
    } catch (err) {
        console.error('\n❌ FAILED! Document NOT found on local DB:', err.message);
        
        // Diagnosi aggiuntiva
        console.log('\n🔍 DIAGNOSTIC INFO:');
        const localInfoAfter = await localDB.info();
        const googleDriveInfoAfter = await googleDriveDB.info();
        
        console.log('After Test:');
        console.log('  Local:', { doc_count: localInfoAfter.doc_count, seq: localInfoAfter.update_seq });
        console.log('  Google Drive:', { doc_count: googleDriveInfoAfter.doc_count, seq: googleDriveInfoAfter.update_seq });
        
        // Verifica che il doc sia effettivamente su Google Drive
        try {
            const verifyDoc = await googleDriveDB.get(testDocId);
            console.log('✅ Document IS on Google Drive:', {
                id: verifyDoc._id,
                rev: verifyDoc._rev
            });
            console.log('❌ BUT NOT replicated to local. Issue: REPLICATION FROM REMOTE NOT WORKING');
        } catch (e) {
            console.error('❌ Document not even on Google Drive. Save failed?');
        }
    }
}

// Esegui il test
await runFullReplicationTest();

---

## 🔧 SOLUZIONI CONSIGLIATE

Basandoti sui risultati del test sopra, ecco i passaggi per risolvere il problema:

### Se il test fallisce (documento non replica from remote):

**Soluzione 1: Aumenta il pollingIntervalMs**
```javascript
pollingIntervalMs: 2000  // Invece di 5000 - controlla più frequentemente
```

**Soluzione 2: Riduci il timeout di replicazione**
```javascript
const repFrom = localDB.replicate.from(googleDriveDB, { 
    live: true, 
    retry: true,
    timeout: 15000  // Aumenta timeout da default 30s
});
```

**Soluzione 3: Aggiungi log nel adapter._changes in adapter.ts**

Modifica `src/adapter.ts` linea ~428 per aggiungere debug:
```typescript
api._changes = function (opts: any): { cancel: () => void } {
    console.log('[_changes] Called with opts:', {
        since: opts.since,
        limit: opts.limit,
        live: opts.live,
        include_docs: !!opts.include_docs
    });
    // ...
```

**Soluzione 4: Verifica che il live listener sia correttamente configurato**

Nel `DriveHandler.load()`, assicurati che il polling chiami `notifyListeners()` ogni volta che rileva cambiamenti:
```typescript
if (metaFile.modifiedTime !== this.metaModifiedTime) {
    this.log('Polling detected change!', metaFile.modifiedTime);
    await this.load();
    this.notifyListeners();  // <-- DEVE ESSERE CHIAMATO
}
```